In [ ]:
import pandas as pd
import os
import multiprocessing as mp
import numpy as np
import subprocess
from pyhandle.handleclient import PyHandleClient
from pprint import pprint

In [ ]:
client = PyHandleClient('rest')

In [ ]:
mip="CMIP"
outpath="/mnt/lustre02/work/ik1017/CMIP6/meta/c34g-qc-prepare/checksumtest/"+"CMIP6/"+mip+"/"
trunk="/mnt/lustre02/work/ik1017/CMIP6/data/"
path=trunk+"CMIP6/"+mip+'/'

In [ ]:
def get_filepid(file):
    try:        
        ncdumpout = subprocess.check_output(['ncdump', '-h', file], shell=False, stderr=subprocess.STDOUT)
        output = str(ncdumpout)
        tid =""
        for item in output.split(';') :
            if 'tracking_id =' in item :
                tid=item
                break
        filepid = tid.split('"')[1].split(':')[1]
        if "21.14100/" not in filepid :
            return ""
        else :
            return filepid
    except:
        return ""

In [ ]:
#get_filepid("/mnt/lustre02/work/ik1017/CMIP6/data/CMIP6/CMIP/MPI-M/MPI-ESM1-2-LR/historical/r1i1p1f1/Amon/tas/gn/v20190710/tas_Amon_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_185001-186912.nc")

In [ ]:
def get_dataset_handle(client, inpid, dataset) :
    try:
        filepidhandle = client.retrieve_handle_record(inpid)
    except:
        print("Handle issue for dataset of file {0}".format(dataset))
        return ""
    version = dataset.split('/')[16]
    try:
        pidlist=filepidhandle["IS_PART_OF"].split(';')
        if len(pidlist) == 1 :
            try:
                pidsolvable=pidlist[0][pidlist[0].find(':')+1:]
                result2 = client.retrieve_handle_record(pidsolvable)
                return result2
            except:
                print("The only dataset handle {0} of file {0} is invalid"
                         .format(pidlist[0],dataset))
                return ""
        else:
            for pid in pidlist :
                try:
                    result2 = client.retrieve_handle_record(pid[pid.find(':')+1:])
                    try:
                        vdid = result2["VERSION_NUMBER"]
                        if version == ("v"+vdid) :
                            return result2
                    except:
                        print("Handle {0} contains no version".format(pid))
                        return ""
                except:
                    print("Handle {0} is invalid".format(pid))
                    return ""
            print("No fitting version for dataset {0}".format(dataset))
            return ""

                
#            while ';' in pidlist :
#                pidlist = pidlist[pidlist.find(';')+1:]
#                print(pidlist)
#            return pidlist
#
#            if ';' in pidlist :
#                return pidlist[0:pidlist.find(';')]
#            else :
#                return pidlist
    except:
        print("No Handle for dataset of file {0}".format(dataset))
        return ""

In [ ]:
def check_checksum(client, dsethandle,dset,filesDifferentOnMistral, filesNotOnMistral, publishReplica) :
    hasDifference=False
    for filepid in dsethandle["HAS_PARTS"].split(';'):
        try:
            filehandle=client.retrieve_handle_record(filepid.split(':')[1])
    #        print(dsethandle["DRS_ID"],filehandle["FILE_NAME"], filehandle["CHECKSUM"], filehandle["FILE_SIZE"])
        except :
            print("PID issue")
            hasDifference = True
            continue
        filepath=dset+'/'+filehandle["FILE_NAME"]
        if not os.path.isfile(filepath) :
            filesNotOnMistral.append(filepath)
            print("No file:"+filepath)
            hasDifference = True
        else :
            sizeOnMistral = subprocess.check_output(['shasum', '-a', "256", filepath], shell=False)
            sizeOnMistral = str(sizeOnMistral).split("'")[1].split(' ')[0]
#            sizeOnMistral = !shasum -a 256 {filepath} | cut -d ' ' -f 1
#            sizeOnMistral = sizeOnMistral[0]
    #        sizeOnMistral = !du -b {trunk+dset.replace('.', '/')} | cut -d $'\t' -f 1
    #        sizeOnMistral = int(sizeOnMistral[0])
    #        sizeOnMistral = !find {trunk+dset.replace('.', '/')} -type f | wc -l
            if sizeOnMistral != filehandle["CHECKSUM"] :
                filesDifferentOnMistral.append(filepath)
                print(sizeOnMistral, filehandle["CHECKSUM"])
                hasDifference = True
    if not hasDifference :
        publishReplica.append('.'.join(dset.split('/')[7:]))

In [ ]:
def writefile(name, var):
    with open(name, 'w') as f:
        for file in var:
            f.write(file+"\n")

In [ ]:
def loop_over_inst(path) :
    global mip, outpath
    client = PyHandleClient('rest')
    file_without_pid=[]
    path_without_files=[]
    filesDifferentOnMistral = []
    filesNotOnMistral = []
    publishReplica = []
    dsetPIDerror = []
    for dirpath, dirnames, filenames in os.walk(path):
        if len(dirnames) == 0 :
            if len(filenames) == 0:
                path_without_files.append(dirpath)
            pid = ""
            for filename in filenames :
                pid = get_filepid(dirpath+"/"+filename)
                if (pid) :
                    break
                else :
                    file_without_pid.append(dirpath+"/"+filename)
            if not pid :
                continue
            dsethandle = get_dataset_handle(client, pid, dirpath)
            if not dsethandle :
                dsetPIDerror.append(dirpath)
                continue
            check_checksum(client, dsethandle, dirpath, filesDifferentOnMistral, filesNotOnMistral, publishReplica)
    writefile(outpath+'fileWithoutPID/'+path.split('/')[len(path.split('/'))-1]+'.txt', file_without_pid)
    writefile(outpath+'pathWithoutFiles/'+path.split('/')[len(path.split('/'))-1]+'.txt', path_without_files)
    writefile(outpath+'filesDifferentOnMistral/'+path.split('/')[len(path.split('/'))-1]+'.txt', filesDifferentOnMistral)
    writefile(outpath+'filesNotOnMistral/'+path.split('/')[len(path.split('/'))-1]+'.txt', filesNotOnMistral)
    writefile(outpath+'publishReplica/'+path.split('/')[len(path.split('/'))-1]+'.txt', publishReplica)
    writefile(outpath+'dsetPIDerror/'+path.split('/')[len(path.split('/'))-1]+'.txt', dsetPIDerror)

In [ ]:
inst = [path + ins for ins in os.listdir(path) if ins not in [ ofile.split('.')[0] for ofile in os.listdir(outpath+'dsetPIDerror/')] ]
print(inst)

In [ ]:
#
#maxworker=mp.cpu_count()
limitworker=20
if len(inst) < limitworker :
    maxworker = len(inst)
else :
    maxworker=limitworker
pool = mp.Pool(maxworker)    

In [ ]:
for direct in [ outpath + subdir for subdir in ["fileWithoutPID", "pathWithoutFiles", "filesDifferentOnMistral", "filesNotOnMistral", "publishReplica", "dsetPIDerror"] ] :
    if not os.path.exists(direct):
        os.makedirs(direct)

In [ ]:
results = [pool.map(loop_over_inst, [path for path in inst])]
#loop_over_inst(path+"MPI-M")

pool.close()